In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random

## Генерация выборок

In [ ]:
def circle(n=1000, noise=0.01):
    radius = 5
    x1 = []
    x2 = []

    def get_circle_label(p, center):
        return 1 if np.linalg.norm([p[0] - center[0], p[1] - center[1]]) < (radius * 0.5) else 0

    for _ in range(n // 2):
        r = np.random.uniform(0, radius * 0.5)
        angle = np.random.uniform(0, 2 * np.pi)
        x = r * np.sin(angle)
        y = r * np.cos(angle)
        noise_x = np.random.uniform(-radius, radius) * noise
        noise_y = np.random.uniform(-radius, radius) * noise
        label = get_circle_label([x + noise_x, y + noise_y], [0, 0])
        point = [x, y]
        x1.append(point) if label == 0 else x2.append(point)

    for _ in range(n // 2):
        r = np.random.uniform(radius * 0.7, radius)
        angle = np.random.uniform(0, 2 * np.pi)
        x = r * np.sin(angle)
        y = r * np.cos(angle)
        noise_x = np.random.uniform(-radius, radius) * noise
        noise_y = np.random.uniform(-radius, radius) * noise
        label = get_circle_label([x + noise_x, y + noise_y], [0, 0])
        point = [x, y]
        x1.append(point) if label == 0 else x2.append(point)

    x1 = np.array(x1)
    x2 = np.array(x2)

    return x1, x2


In [ ]:
def xor(n=200, noise=0.1):
    def get_xor_label(p):
        return 1 if p[0] * p[1] >= 0 else 0

    x1 = []
    x2 = []
    for _ in range(n):
        x = np.random.uniform(-5, 5)
        padding = 0.3
        x += padding if x > 0 else -padding
        y = np.random.uniform(-5, 5)
        y += padding if y > 0 else -padding
        noise_x = np.random.uniform(-5, 5) * noise
        noise_y = np.random.uniform(-5, 5) * noise
        label = get_xor_label([x + noise_x, y + noise_y])
        x1.append([x, y]) if label == 0 else x2.append([x,y])

    x1 = np.array(x1)
    x2 = np.array(x2)

    return x1, x2

In [ ]:
def gaussian(n_points=200, noise=0.1):
    center1 = np.array([1, 1])
    center2 = np.array([-1, -1])
    radius = 0.3*noise

    X1 = np.random.randn(n_points // 2, 2) * radius + center1
    X2 = np.random.randn(n_points // 2, 2) * radius + center2

    return X1 + np.random.uniform(-noise, noise, X1.shape), X2 + np.random.uniform(-noise, noise, X2.shape)

In [ ]:
def spiral(n_points=100, noise=0.2):
    n_points //= 2
    n = np.sqrt(np.random.rand(n_points, 1)) * 720 * (2 * np.pi) / 360
    d1x = -np.cos(n) * n + np.random.rand(n_points, 1) * noise
    d1y = np.sin(n) * n + np.random.rand(n_points, 1) * noise

    d2x = np.cos(n) * n + np.random.rand(n_points, 1) * noise
    d2y = -np.sin(n) * n + np.random.rand(n_points, 1) * noise

    return np.hstack((d1x, d1y)),  np.hstack((d2x, d2y))

In [ ]:
def split_array(array, ratio):
    split_idx = int(len(array) * ratio / 100)
    return array[:split_idx], array[split_idx:]

In [ ]:
def get_dataset_without_separation(dataset_name, number_of_samples, noise):
    if dataset_name == "circle":
        x1, x2 = circle(number_of_samples, noise)
        features = np.vstack((x1, x2))
        targets = np.hstack((np.zeros(len(x1)), np.ones(len(x2))))
    elif dataset_name == "xor":
        x1, x2 = xor(number_of_samples, noise)
        features = np.vstack((x1, x2))
        targets = np.hstack((np.zeros(len(x1)), np.ones(len(x2))))
    elif dataset_name == "gaussian":
        x1, x2 = gaussian(number_of_samples, noise)
        features = np.vstack((x1, x2))
        targets = np.hstack((np.zeros(len(x1)), np.ones(len(x2))))
    elif dataset_name == "spiral":
        x1, x2 = spiral(number_of_samples, noise)
        features = np.vstack((x1, x2))
        targets = np.hstack((np.zeros(len(x1)), np.ones(len(x2))))
    else:
        raise ValueError("Unknown dataset name")

    combined = list(zip(features, targets))
    random.shuffle(combined)

    return combined

def get_dataset(dataset_name, number_of_samples, ratio, noise):
    combined = get_dataset_without_separation(dataset_name, number_of_samples, noise)

    trains, tests = split_array(combined, ratio)

    x_train, y_train = zip(*trains)
    x_test, y_test = zip(*tests)
    x_train = np.array(x_train)
    y_train = np.array(y_train)
    x_test = np.array(x_test)
    y_test = np.array(y_test)

    return x_train, y_train, x_test, y_test

## Функция отображения выборки

In [ ]:
def print_dataset(x1,x2):
  plt.figure(figsize=(8, 8))
  plt.scatter(x1[:, 0], x1[:, 1], c='b', label='Class blue')
  plt.scatter(x2[:, 0], x2[:, 1], c='r', label='Class red')
  plt.title('Generated Data')
  plt.legend()
  plt.grid(True)
  plt.show()

### Пример генерации выборки

In [ ]:
x1, x2 = xor(1000, 0.1)

# print_dataset(x1,x2)

## Вводные данные

In [ ]:
import torch
from sklearn.metrics import confusion_matrix
from torch import nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.data import random_split
#!pip install torch torchvision torchaudio -f https://download.pytorch.org/whl/torch_stable.html
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
seed = 21210
random.seed(seed)
torch.manual_seed(seed)
np.random.seed(seed)

ratio = 50
eps0 = 0.01

curr_dataset = "circle"
number_of_samples = 50

input_size = 2
output_size = 1

hidden_layers = 5
num_neurons = 5
activation_function = 'tanh'

epochs = 100
learning_rate = 0.03

## Создаём скелет нейронной сети с помощью nn.Module

In [ ]:
class MLP(nn.Module):
    def __init__(self, input_size, hidden_layers, num_neurons, output_size, activation_function):
        super(MLP, self).__init__()

        # Определение функции активации
        if activation_function == 'sigmoid':
            activation = nn.Sigmoid()
        elif activation_function == 'tanh':
            activation = nn.Tanh()
        elif activation_function == 'relu':
            activation = nn.ReLU()
        else:
            raise ValueError("Неизвестная функция активации")

        # Список слоёв
        self.layers = nn.ModuleList()

        # Добавление входного слоя
        self.layers.append(nn.Linear(input_size, num_neurons))
        self.layers.append(activation)

        # Добавление скрытых слоёв с нашей функцией активации
        for _ in range(hidden_layers):
            self.layers.append(nn.Linear(num_neurons, num_neurons))
            self.layers.append(activation)

        # Добавление выходного слоя с функцией активацией сигмоид
        self.layers.append(nn.Linear(num_neurons, output_size))
        self.layers.append(nn.Sigmoid())

    # проходим по всем слоям и вычисляем значения
    def forward(self, x):
      for layer in self.layers:
        x = layer(x)
      return x

## Обучение

In [ ]:
def fit(model, features, targets, test_features, test_targets, epochs, statistic=True):
    features_tensor=torch.from_numpy(features).type(torch.FloatTensor)
    targets_tensor=torch.from_numpy(targets).type(torch.FloatTensor)

    error_function = torch.nn.BCELoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

    for epoch in range(epochs):
        for feature, target in zip(features_tensor, targets_tensor):
            model.train()   # тренируем

            y_pred = model(feature).squeeze()
            error = error_function(y_pred, target)

            optimizer.zero_grad()
            error.backward()
            optimizer.step()

        if (statistic):
            model.eval()
            with torch.inference_mode():
                test_features_tensor = torch.tensor(test_features).type(torch.FloatTensor)
                test_predict = model(test_features_tensor).squeeze()
                test_targets_tensor = torch.tensor(test_targets).type(torch.FloatTensor)
                test_error = error_function(test_predict, test_targets_tensor)

                # преобразовать предсказания в бинарные
                binary_pred = torch.round(test_predict)
                # считаем количество правильных
                good_predict = torch.sum(torch.eq(test_targets_tensor, binary_pred))
                total_count = test_targets_tensor.numel()
                accuracy = good_predict / total_count

            if epoch % 10 == 0:
                print(f'Epoch {epoch}, error: {error}, test error: {test_error}, accuracy: {accuracy}')

        if (not statistic): model.eval()

In [ ]:
model = MLP(input_size, hidden_layers, num_neurons, output_size, activation_function)

features, targets, test_features, test_targets = get_dataset(curr_dataset, number_of_samples, ratio, eps0)

fit(model, features, targets, test_features, test_targets, epochs)

Epoch 0, error: 0.5729085803031921, test error: 0.6863927245140076, accuracy: 0.5600000023841858
Epoch 10, error: 0.8083645701408386, test error: 0.712940514087677, accuracy: 0.4399999976158142
Epoch 20, error: 0.8249736428260803, test error: 0.7202110886573792, accuracy: 0.4399999976158142
Epoch 30, error: 0.7938252687454224, test error: 0.725989580154419, accuracy: 0.4399999976158142
Epoch 40, error: 0.4961720407009125, test error: 0.8133820295333862, accuracy: 0.4399999976158142
Epoch 50, error: 0.10681627690792084, test error: 0.9675382375717163, accuracy: 0.6399999856948853
Epoch 60, error: 0.034751277416944504, test error: 1.1609948873519897, accuracy: 0.6399999856948853
Epoch 70, error: 0.01740121655166149, test error: 1.360985279083252, accuracy: 0.6399999856948853
Epoch 80, error: 0.011487836018204689, test error: 1.4897569417953491, accuracy: 0.6399999856948853
Epoch 90, error: 0.008542420342564583, test error: 1.5840438604354858, accuracy: 0.6399999856948853


# Кросс - Валидация

### Генерируем и разделяем выборку на k частей

In [ ]:
def get_dataset_for_validation(dataset_name, number_of_samples, eps0, k):
    combined = get_dataset_without_separation(dataset_name, number_of_samples, eps0)
    x, y = zip(*combined)

    x1 = [part[0] for part in x]
    x2 = [part[1] for part in x]

    x1_part = np.array_split(np.array(x1), k)
    x2_part = np.array_split(np.array(x2), k)

    feature_parts = [np.column_stack((x1_part[i], x2_part[i])) for i in range(k)]
    target_parts = np.array_split(np.array(y), k)

    return np.array(feature_parts), np.array(target_parts)

def separation_dataset(S, exclude_index):
    if (exclude_index >= S.shape[0]):
        print("Неправильный индекс")
        exit()

    indices_to_combine = [i for i in range(S.shape[0]) if i != exclude_index]

    # Объеденение выбранных массивов
    combined_array = np.concatenate(S[indices_to_combine])

    return np.array(combined_array), np.array(S[exclude_index])

In [ ]:
curr_num_neurons = 5
curr_input_size = 2
curr_output_size = 1
min_num_layers = 2
max_num_layers = 4
num_epochs = 100
num_samples_for_cross_valid = 500
curr_dataset_for_cross_valid = 'circle'
curr_error_function = torch.nn.BCELoss()

k = 5

In [ ]:
models_for_cross_valid = list()
activation_functuions = ['sigmoid', 'tanh', 'relu']
statistic_info = list()
# генерируем модели с разными функциями активации и разным кол-вом скрытых слоев

for curr_num_hidden_layers in range(2, 4 + 1):
    for curr_activ_func in activation_functuions:
        models_for_cross_valid.append(MLP(curr_input_size, curr_num_hidden_layers, curr_num_neurons, curr_output_size, curr_activ_func))
        statistic_info.append([curr_num_hidden_layers, curr_activ_func])

feature_parts, target_parts = get_dataset_for_validation(curr_dataset_for_cross_valid, num_samples_for_cross_valid, eps0, k)

for exclude_index in range(k):
    train_features, test_features = separation_dataset(feature_parts, exclude_index)
    train_targets, test_targets = separation_dataset(target_parts, exclude_index)

    #print(f'########## {exclude_index} теперь явялется тестовой выборкой ###########')

    for i in range(len(models_for_cross_valid)):
        curr_model = models_for_cross_valid[i]
        optimizer = torch.optim.SGD(curr_model.parameters(), lr=learning_rate)

        fit(curr_model, train_features, train_targets, test_features, test_targets, num_epochs, statistic=False)

        test_features_tensor = torch.tensor(test_features).type(torch.FloatTensor)
        test_pred = curr_model(test_features_tensor).squeeze()
        test_features_tensor = torch.tensor(test_features).type(torch.FloatTensor)

        test_targets_tensor = torch.tensor(test_targets).type(torch.FloatTensor)
        test_error = curr_error_function(test_pred, test_targets_tensor)
        print(f"Модель с кол-вом скрытых слоёв: {statistic_info[i][0]} и функцией активацией: {statistic_info[i][1]}")
        print(f"test error {test_error}")

Модель с кол-вом скрытых слоёв: 2 и функцией активацией: sigmoid
test error 0.02289244905114174
Модель с кол-вом скрытых слоёв: 2 и функцией активацией: tanh
test error 0.013675231486558914
Модель с кол-вом скрытых слоёв: 2 и функцией активацией: relu
test error 0.00021143973572179675
Модель с кол-вом скрытых слоёв: 3 и функцией активацией: sigmoid
test error 0.6954557299613953
Модель с кол-вом скрытых слоёв: 3 и функцией активацией: tanh
test error 0.005780908279120922
Модель с кол-вом скрытых слоёв: 3 и функцией активацией: relu
test error 0.00022160203661769629
Модель с кол-вом скрытых слоёв: 4 и функцией активацией: sigmoid
test error 0.6965278387069702
Модель с кол-вом скрытых слоёв: 4 и функцией активацией: tanh
test error 0.00022325647296383977
Модель с кол-вом скрытых слоёв: 4 и функцией активацией: relu
test error 0.6942646503448486
Модель с кол-вом скрытых слоёв: 2 и функцией активацией: sigmoid
test error 0.004596631973981857
Модель с кол-вом скрытых слоёв: 2 и функцией акти